In [1]:
import pandas as pd
from pulp import *

In [2]:
filepath = r"C:\Users\lbetham\GitHub\Fantasy-Premier-League\data"
cleaned_players_df = pd.read_csv(filepath + r"\2022-23\cleaned_players.csv", low_memory=False)
teams_df = pd.read_csv(filepath + r"\2022-23\teams.csv", low_memory=False)
past_seasons_df = pd.read_csv(filepath + r"\cleaned_merged_seasons.csv", low_memory=False)
raw_players_df = pd.read_csv(filepath + r"\2022-23\players_raw.csv", low_memory=False)

In [3]:
model_df = pd.merge(cleaned_players_df,raw_players_df[['first_name','second_name','team_code']],"left",on=['first_name','second_name'])
model_df = pd.merge(model_df,teams_df[['code','name']],"left",left_on='team_code',right_on='code')
model_df.rename({'name':'team_name','code':'team_code'},axis=1,inplace=True)

In [4]:
model_df["full_name"] = model_df["first_name"] + " " + model_df["second_name"]

In [5]:
model_df.head(2)

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,...,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type,team_code,team_code,team_name,full_name
0,Cédric,Alves Soares,1,1,48,1481,27,327.1,318.4,111.0,...,3,0,3,0.3,45,DEF,3,3,Arsenal,Cédric Alves Soares
1,Bernd,Leno,0,0,10,360,9,0.0,85.0,0.0,...,1,0,0,0.6,45,GK,3,3,Arsenal,Bernd Leno


In [6]:
# Helper variables
POS = model_df.element_type.unique()
CLUBS = model_df.team_name.unique()
BUDGET = 1000
pos_available = {
    'DEF': 5,
    'FWD': 3,
    'MID': 5,
    'GK': 2,
}

# Initialize Variables
names = model_df.full_name.tolist()
teams = model_df.team_name.tolist()
positions = model_df.element_type.tolist()
prices = model_df.now_cost.tolist()
points = model_df.total_points.tolist()
player_ids = [LpVariable("player_" + str(i), cat="Binary") for i in model_df.index]

In [7]:
# Initialize the problem
prob = LpProblem("FPL_Player_Choices", LpMaximize)

In [8]:
# Define the objective
prob += lpSum(player_ids[i] * points[i] for i in range(len(model_df))) # Objective

In [9]:
# Build the constraints
prob += lpSum(player_ids[i] * model_df.now_cost[model_df.index[i]] for i in range(len(model_df))) <= BUDGET # Budget Limit

for pos in POS:
  prob += lpSum(player_ids[i] for i in range(len(model_df)) if positions[i] == pos) <= pos_available[pos] # Position Limit

for club in CLUBS:
  prob += lpSum(player_ids[i] for i in range(len(model_df)) if teams[i] == club) <= 3 # Club Limit

In [10]:
# Solve the problem
prob.solve()

1

In [11]:
for v in prob.variables():
  if v.varValue != 0:
    name = model_df.full_name[int(v.name.split("_")[1])]
    club = model_df.team_name[int(v.name.split("_")[1])]
    position = model_df.element_type[int(v.name.split("_")[1])]
    point = model_df.total_points[int(v.name.split("_")[1])]
    price = model_df.now_cost[int(v.name.split("_")[1])]
    print(name, position, club, point, price, sep=" | ")

Bukayo Saka | MID | Arsenal | 179 | 80
Gabriel dos Santos Magalhães | DEF | Arsenal | 146 | 50
James Maddison | MID | Leicester | 181 | 80
Virgil van Dijk | DEF | Liverpool | 183 | 65
Alisson Ramses Becker | GK | Liverpool | 176 | 55
Trent Alexander-Arnold | DEF | Liverpool | 208 | 75
João Cancelo | DEF | Man City | 201 | 70
Bernardo Veiga de Carvalho e Silva | MID | Man City | 155 | 70
Matty Cash | DEF | Aston Villa | 147 | 50
James Ward-Prowse | MID | Southampton | 159 | 65
Michail Antonio | FWD | West Ham | 140 | 75
Jarrod Bowen | MID | West Ham | 206 | 85
José Malheiro de Sá | GK | Wolves | 146 | 50
Ivan Toney | FWD | Brentford | 139 | 70
Bryan Mbeumo | FWD | Brentford | 119 | 60
